# Motivating example: Parallel image processing with scikit-image

To get a sense of what IPython.parallel might be used for,
we start with an example of some batch processing of image files with [scikit-image](http://scikit-image.org/).
We will revisit pieces of this example as we learn about the different components of IPython.

In [ ]:
%run ../hubble-tiles.py

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import sys,os,re,time
import urllib

import numpy as np

import ipyparallel as ipp

In [ ]:
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import blob_doh

Define a function to find the blobs in an image:

In [ ]:
def find_blobs(img_path):
    image = imread(img_path)
    image_gray = rgb2gray(image)
    return image, blob_doh(image_gray, max_sigma=30, threshold=.01)

and show the blobs on the image with matplotlib:

In [ ]:
def plot_blobs(image, blobs):
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.imshow(image, interpolation='nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color='r', linewidth=2, fill=False)
        ax.add_patch(c)

Get our list of picture files:

In [ ]:
ls

In [ ]:
import os
pictures_dir = os.path.join('..', 'hubble-400x400')

pictures = []
for directory, subdirs, files in os.walk(pictures_dir):
    for fname in files:
        pictures.append(os.path.join(directory, fname))

pictures[:5]

Let's test our function locally, to see what it does.

In [ ]:
for p in pictures[:3]:
    img, blobs = find_blobs(p)
    plot_blobs(img, blobs)

# Now in parallel

First, we connect our parallel Client

In [ ]:
rc = ipp.Client()
all_engines = rc[:]
view = rc.load_balanced_view()

Then we initialize the namespace on all of the engines with imports

In [ ]:
%px import os; os.chdir("{os.getcwd()}")

In [ ]:
%%px

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import blob_doh

and make sure some functions are defined everywhere (this is only necessary for the `contours_in_url` case)

Now we can iterate through all of our pictures, and detect and display any corners we find

In [ ]:

amr = view.map_async(get_blobs, pictures[:20], ordered=False)
for img, blobs in amr:
    plot_blobs(img, blobs)
